# Import module

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

# Cinema infomation webcraw

In [2]:
def dataset_webcraw(url):
    response = requests.get(url)
    res = BeautifulSoup(response.text) # 預設html.parser，不打也相同
    latest_data = res.find_all('div', class_ = 'download-item')[-1].a['href']
    return latest_data
url = 'https://data.gov.tw/dataset/22213'
latest_data = dataset_webcraw(url)

In [ ]:
df = pd.read_csv(latest_data)
cinema_data = df[['事業名稱','地址']]
cinema_data.dropna(inplace=True)

# Save to DataBase

In [6]:
# 帶index
conn = sqlite3.connect('TW_cinema_info.db')
cinema_data.to_sql('TW_cinema_info_latest', con=conn, if_exists='replace', index=True)
conn.close()